# Library loading

In [1]:
%matplotlib inline

import pandas as pd # manipulate dataframes
import matplotlib
import matplotlib.pyplot as plt # plotting
import numpy as np

import time, h5py, imelt, torch

from sklearn.metrics import mean_squared_error

from tqdm import tqdm 

# First we check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

from ax.plot.contour import plot_contour
from ax.plot.slice import plot_slice
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render
from ax.utils.tutorials.cnn_utils import train, evaluate

# Fixing random seeds for reproducibility
np.random.seed = 167 # fix random seed for reproducibility

cuda


# Training functions

In [17]:
def net_train(net, dataset, parameterization, dtype, device):
    # handling the network
    net.output_bias_init() # we initialize the output bias
    net.to(dtype=dtype, device=device) # set dtype and send to device

    # Define loss and optimizer
    criterion = torch.nn.MSELoss() # the criterion : MSE
    criterion.to(device)
    
    optimizer = torch.optim.Adam(net.parameters(), 
                                 lr=parameterization.get("lr", 0.001), # 0.001 is used if no lr is specified
                                 weight_decay=parameterization.get("weight_decay", 0.0)) # L2 loss
                
    # training
    net, record_train_loss, record_valid_loss = imelt.training2(net,dataset,criterion,optimizer,
                                                                         save_switch=False,
                                                                         nb_folds=parameterization.get("nb_folds", 20),
                                                                         train_patience=parameterization.get("patience", 10),
                                                                         min_delta=parameterization.get("min_delta", 0.05),
                                                                         verbose=False)

    # to avoid any problem with CUDA memory...
    # del neuralmodel, criterion
    # torch.cuda.empty_cache()
    
    return net, record_train_loss, record_valid_loss

def init_net(parameterization, dataset):

    # get hyperparameters for the network 
    nb_neurons = parameterization.get("nb_neurons", 300)
    nb_layers = parameterization.get("nb_layers", 3)
    p_drop = parameterization.get("p_drop", 0.05)
    
    # model declaration
    model = imelt.model(4,nb_neurons,nb_layers,dataset.nb_channels_raman, p_drop) # declaring model
                                  
    return model # return untrained model

def train_evaluate(parameterization):

    # Get neural net
    untrained_net = init_net(parameterization, dataset) 
    
    # train
    trained_net, record_train_loss, record_valid_loss = net_train(untrained_net, dataset, 
                            parameterization, dtype=dtype, device=device)
    
    output = {"loss_train": (record_train_loss[-1], 0.0), "loss_valid": (record_valid_loss[-1], 0.0)}
    
    # return the accuracy of the model as it was trained in this run
    return output

# Launching the BO experiment

In [18]:
#torch.cuda.set_device(0) #this is sometimes necessary for me
dtype = torch.float
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# custom data loader, automatically sent to device
dataset = imelt.data_loader("./data/NKAS_viscosity_reference.hdf5",
                         "./data/NKAS_Raman.hdf5",
                         "./data/NKAS_density.hdf5",
                         "./data/NKAS_optical.hdf5",
                         device)

best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "nb_neurons", "type": "range", "bounds": [100, 500]},
        {"name": "nb_layers", "type": "range", "bounds": [2, 8]},
        {"name": "lr", "type": "range", "bounds": [1e-5, 0.001]}, # 0.008
        {"name": "nb_folds", "type": "range", "bounds": [2, 30]}, # 0.008
        #{"name": "min_delta", "type": "range", "bounds": [0.01, 1.0]},
        #{"name": "patience", "type": "range", "bounds": [10, 500]},
        #{"name": "weight_decay", "type": "range", "bounds": [1.0e-5, 1.0e-2]},        
    ],
    total_trials = 100,
    evaluation_function=train_evaluate,
    objective_name='loss_valid',
)

print(best_parameters)
means, covariances = values
print(means)
print(covariances)
#view raw


[INFO 06-22 18:06:22] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter nb_neurons. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-22 18:06:22] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter nb_layers. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-22 18:06:22] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-22 18:06:22] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter nb_folds. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dic

[INFO 06-22 18:14:12] ax.service.managed_loop: Running optimization trial 16...
[INFO 06-22 18:14:24] ax.core.experiment: Attached data has some metrics ({'loss_train'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[INFO 06-22 18:14:24] ax.service.managed_loop: Running optimization trial 17...
[INFO 06-22 18:14:32] ax.core.experiment: Attached data has some metrics ({'loss_train'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[INFO 06-22 18:14:32] ax.service.managed_loop: Runni

[INFO 06-22 18:24:47] ax.service.managed_loop: Running optimization trial 34...
[INFO 06-22 18:25:25] ax.core.experiment: Attached data has some metrics ({'loss_train'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[INFO 06-22 18:25:25] ax.service.managed_loop: Running optimization trial 35...
[INFO 06-22 18:25:57] ax.core.experiment: Attached data has some metrics ({'loss_train'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[INFO 06-22 18:25:57] ax.service.managed_loop: Runni

[INFO 06-22 18:47:58] ax.service.managed_loop: Running optimization trial 52...
[INFO 06-22 18:48:41] ax.core.experiment: Attached data has some metrics ({'loss_train'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[INFO 06-22 18:48:41] ax.service.managed_loop: Running optimization trial 53...
[INFO 06-22 18:49:43] ax.core.experiment: Attached data has some metrics ({'loss_train'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[INFO 06-22 18:49:43] ax.service.managed_loop: Runni

{'nb_neurons': 461, 'nb_layers': 2, 'lr': 0.0006179899391140026, 'nb_folds': 2}
{'loss_valid': 39.316041493612765, 'loss_train': 21.28754931464348}
{'loss_valid': {'loss_valid': 1.720429152784467e-06, 'loss_train': 0.0}, 'loss_train': {'loss_valid': 0.0, 'loss_train': 7.18096285017969e-07}}


In [21]:
render(plot_contour(model=model, param_x='nb_layers', param_y='nb_neurons', metric_name='loss_valid'))

In [70]:
render(plot_contour(model=model, param_x='nb_neurons', param_y='p_drop', metric_name='loss_valid'))

In [75]:
render(plot_slice(model, "p_drop", "loss_train"))